# NB #1:  Pool Exploration 

### Pool address: 0x8b6e6E7B5b3801FEd2CaFD4b22b8A16c2F2Db21a  -- 80% WETH - 20% DAI 

The Balancer Pool cadCAD model provides a core infrastructure for simulating Balancer Pools in the **Token Engineering Design and Verification process**.

It allows to simulate Balancer Pool transactions and verify new designs and application cases.  
In this notebook we plug **historical on-chain data** to a cadCAD model, and analyse **Pool Power** and **Pool Characteristics**.

For more information check out the [Balancer Simulations documentation](https://token-engineering-balancer.gitbook.io/balancer-simulations/).

# A. System Context

### A1.1 System Specification
System specification details are available in the Balancer Simulations documentation:
- [Differential Specification](https://token-engineering-balancer.gitbook.io/balancer-simulations/balancer-simulations/v10nboverview)
- [Mathematical Specification](https://token-engineering-balancer.gitbook.io/balancer-simulations/additional-code-and-instructions/balancer-the-python-edition/balancer_math.py)
- [Model Architecture](https://token-engineering-balancer.gitbook.io/balancer-simulations/balancer-simulations/v10nboverview)
- [Naming Convention](https://token-engineering-balancer.gitbook.io/balancer-simulations/additional-code-and-instructions/naming-convention)

# B. cadCAD Simulations

### B1.1 Dependencies

In [ ]:
import pandas as pd 
from cadCAD.configuration.utils import config_sim

### B1.2 Initialize Pool


In this section you specify the Balancer Pool to be analyzed and run scripts to produce the'genesis_state', based on on-chain data. 

More information here: [Balancer Simulations documentation](https://token-engineering-balancer.gitbook.io/balancer-simulations/additional-code-and-instructions/onchaintransactions).  

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

from decimal import Decimal

# Spot price reference must be a symbol of a token in the pool in ALL_CAPS, you can ignore the spot price parameter for the simulations in this notebook.
parameters = {
    'spot_price_reference': ['DAI'],
    'decoding_type': ['CONTRACT_CALL']
}


#### Import Pool Transactions and Initialize Pool state

Genesis state, pool transactions, and (external) USD token prices are pulled from on-chain/API data collected in a .json file.  
**Choose *your* Balancer Pool, and produce this .json file first, then add the file path below.**

Learn more how to create this file using parsing scripts in the [documentation](https://token-engineering-balancer.gitbook.io/balancer-simulations/v/master/additional-code-and-instructions/onchaintransactions).


In [ ]:
from model.genesis_states import generate_initial_state

initial_values = generate_initial_state(initial_values_json='data/0x8b6e6e7b5b3801fed2cafd4b22b8a16c2f2db21a-initial_pool_states-prices.json', spot_price_base_currency=parameters['spot_price_reference'][0])

#### State Variables and Initial Values

In [ ]:
print('## State Variables')
print('# Pool')
pool = initial_values['pool']
pp.pprint(initial_values)


#### External USD Token Prices

In [ ]:
print('# External token prices, initial state')
token_prices = initial_values['token_prices']
pp.pprint(token_prices)

print('# Action Type')
action_type = initial_values['action_type']
pp.pprint(action_type)


### B1.3 State Update Functions & Policies

Balancer Simulations replicate Balancer Pool Transactions in state update functions.  
For a detailed description, please visit the [Balancer Simulations documentation](https://token-engineering-balancer.gitbook.io/balancer-simulations/additional-code-and-instructions/balancer-the-python-edition).

### B1.4 Partial State Update Blocks

Partial State Update Blocks combine the following steps:  
1. **Initialize pool**, generate genesis state
2. Compute subsequent **actions in discrete timesteps**, store datetime and update the pool state variables
3. Update external **USD prices in discrete timesteps**, and store datetime  

For more information please visit the [Balancer Simulations documentation/Model Overview](https://token-engineering-balancer.gitbook.io/balancer-simulations/v/master/balancer-simulations/v10nboverview).  

To inject historical on-chain transactions to the model, reference the **actions.json of your pool below**.

In [ ]:
from model.partial_state_update_block import generate_partial_state_update_blocks

result = generate_partial_state_update_blocks('data/0x8b6e6e7b5b3801fed2cafd4b22b8a16c2f2db21a-actions-prices.json')
partial_state_update_blocks = result['partial_state_update_blocks']
pp.pprint(partial_state_update_blocks)

### B1.5 Configuration

[cadCAD simulations](https://github.com/cadCAD-org/cadCAD/blob/master/documentation/README.md) support Monte Carlo runs, and parameter sweeps which are not applied in this notebook and can be ignored here.  
Timesteps 'T' are defined by your pool's .json file, no need to make edits here.

In [ ]:
steps_number = result['steps_number']
print('# Steps ', steps_number)
sim_config = config_sim(
    {
        'N': 1,  # number of monte carlo runs
        'T': range(steps_number - 1),  # number of timesteps
        'M': parameters,  # simulation parameters
    }
)

### B1.6 Execution

In [ ]:
from model.sim_runner import *

df = run(initial_values, partial_state_update_blocks, sim_config)

### B1.7 Simulation Output Preparation

Post-processing (utils.py) splits up state variable dictionaries, and adds metrics to the data frame, such as 
- `token_total_value` (Total Value Locked in the pool)
- `invariant` (Invariant V of the pool)
- `generated_fees_(tokensymbol)` (fee collected per transaction)
- `token_k_values`  (token value in USD)  
based on  
- `token_k_balances`
- `token_k_price`  (external price feed)

In [ ]:
from model.parts.utils import post_processing

p_df = post_processing(df, include_spot_prices=False)

p_df

# C. Simulation Outcome & Pool Exploration

The plots below offer some keys metrics for pool analysis.

**a) Pool Energy:**  
- C1.1 TVL & Invariant V
- C1.2 Token Balances (individual token balances, over time)

**b) Pool Characteristics:**  
- C1.3 Sources of Growth (BPT, Fees)
- C1.4 Action Types

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
print("Observation Time")
print(f"Start: {p_df['change_datetime'].min()}")
print(f"End: {p_df['change_datetime'].max()}")
print(f"Total Observation Period: {p_df['change_datetime'].max() - p_df['change_datetime'].min()}")
print("\n")
print(f"Total No. of Timesteps (incl. Price Updates): {(p_df.iloc[-1]['timestep'])}")
print(f"Total No. of Transactions: {len(p_df[p_df.action_type != 'external_price_update'])}")

### C1.1 TVL & Invariant V

In [ ]:
#TVL vs. Invariant V (total observation period)
fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.add_trace(go.Scatter(x=p_df.timestep,y=p_df.tvl, line=dict(color='#f36315'), name="TVL"), secondary_y=False)
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.invariant, line=dict(color='#2C1839'), name="Invariant V"), secondary_y=True)
fig.update_layout(title_text='<b>TVL vs. Invariant V</b>')
fig.update_xaxes(title_text='timestep')
fig.update_yaxes(title_text='<b>TVL</b> in USD', secondary_y=False)
fig.update_yaxes(title_text='<b>Invariant V', secondary_y=True, range=[183000, 197000])
fig.show()  

In [ ]:
# Min/Max overall (total observation period)
print("TVL min/max")
print(f"Min: {p_df['tvl'].min()} at timestep {p_df['tvl'].argmin()}")
print(f"Max: {p_df['tvl'].max()} at timestep {p_df['tvl'].argmax()}")
print("\n")
print(f"Invariant V min/max")
print(f"Min: {p_df['invariant'].min()} at timestep {p_df['invariant'].argmin()}")
print(f"Max: {p_df['invariant'].max()} at timestep {p_df['invariant'].argmax()}")

In [ ]:
# Min/Max before LP exit

#define observation period before LP exited
rp_df = p_df.loc[0:44000,:]

print("TVL min/max BEFORE LP EXIT")
print(f"Min: {rp_df['tvl'].min()} at timestep {rp_df['tvl'].argmin()}")
print(f"Max: {rp_df['tvl'].max()} at timestep {rp_df['tvl'].argmax()}")
print("\n")
print(f"Invariant V min/max BEFORE LP EXIT")
print(f"Min: {rp_df['invariant'].min()} at timestep {rp_df['invariant'].argmin()}")
print(f"Max: {rp_df['invariant'].max()} at timestep {rp_df['invariant'].argmax()}")

**Observation:**
- after timestep 44.000 the pool lost almost 100% of its value, since liquidity providers exited
- before LP exited, TVL grew significantly, mainly driven by WETH value growth (and not growth of the pool itself)
- during this period, the actual "pool energy" represented by the Invariant V increased only moderately

### C1.2 Token Balances

In [ ]:
#balance growth % (total observation period)
dai_growth = ((p_df.iloc[-1][['token_dai_balance']])-(p_df.iloc[0][['token_dai_balance']]))*100/(p_df.iloc[0][['token_dai_balance']])
weth_growth = ((p_df.iloc[-1][['token_weth_balance']])-(p_df.iloc[0][['token_weth_balance']]))*100/(p_df.iloc[0][['token_weth_balance']])
growth = list(zip(dai_growth, weth_growth)) 
g_df = pd.DataFrame(growth, columns = ['dai_growth','weth_growth' ]).transpose(copy=True).reset_index()
g_df.columns =['token', 'growth']
print(g_df)
#plot
fig = make_subplots(rows=2, cols=2, subplot_titles=('DAI Growth', 'Balance Growth in %', 'WETH Growth'))
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.token_dai_balance, line=dict(color='#4675ed'), name='token_dai_balance'), row=1, col=1)
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.token_weth_balance, line=dict(color='#4145ab'), name='token_weth_balance'), row=2, col=1)
fig.add_trace(go.Bar(y=g_df.growth, x=g_df.token, marker_color=['#4675ed','#4145ab']), row=1, col=2)
fig.update_layout(height=600, width=1000, showlegend=False, title_text='<b>Balance Growth</b> in observation period')
fig.show()

### C1.3 Sources of Growth 

In [ ]:
#Pool Shares (total observation period)
fig = px.line(p_df, x=p_df.timestep,y=p_df.pool_shares)
fig.update_layout(height=300, width=1000, title_text='<b>Pool Shares (BPT)</b>')
fig.update_xaxes(title_text='timestep')
fig.update_yaxes(title_text='<b>pool shares</b> in #', range=[-0.05,102.00])
fig.show()

In [ ]:
#Fees (total observation period)
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.generated_fees_dai, line=dict(color='#4675ed'), name='fees DAI'), row=1, col=1)
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.generated_fees_weth, line=dict(color='#4145ab'), name='fees WETH'), row=2, col=1)
fig.update_yaxes(title_text='token amount', secondary_y=False)
fig.update_layout(height=500, width=1000, title_text='<b>Fees Generated</b> per transaction')

In [ ]:
# DAI fee events vs. ETH fee events (total observation period)
WETH_fee_events = len(p_df[p_df.generated_fees_weth > 0])
DAI_fee_events = len(p_df[p_df.generated_fees_dai > 0])
values = [DAI_fee_events, WETH_fee_events]
labels = ['DAI fees', 'WETH fees']
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_traces(hoverinfo='label+percent', marker=dict(colors=['#4675ed','#4145ab']))
fig.update_layout(height=500, width=1000, title_text='<b>Fee generating events</b> in total observation time')
fig.show()

In [ ]:
#proportions of fee value contribution, in fixed USD value (total observation period)
daif = p_df.groupby('action_type').sum()['generated_fees_dai'].drop(['exit', 'join', 'external_price_update','pool_creation'])
wethf = p_df.groupby('action_type').sum()['generated_fees_weth'].drop(['exit', 'join', 'external_price_update','pool_creation'])
fee_v = pd.concat([daif, wethf], axis=1)

#calculate USD value (define USD value, in this case locked to initial state observation time)
fee_v['DAI_fee_value'] = fee_v.generated_fees_dai*1.0053414509361551 #define DAI price
fee_v['WETH_fee_value'] = fee_v.generated_fees_weth*594.3526451552318 #define WETH price

#add total 
fee_v = fee_v.append(fee_v.sum().rename('total'))
print(fee_v)

#plot pie chart comparing ETH fees vs DAI fees
values = [fee_v.loc['total']['DAI_fee_value'], fee_v.loc['total']['WETH_fee_value']] 
labels = ['DAI_fees', 'WETH_fees']
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_traces(hoverinfo='label+percent', marker=dict(colors=['#4675ed','#4145ab']))
fig.update_layout(height=500, width=1000, title_text="<b>Fee value contribution</b> in USD, in total observation time")
fig.show()

In [ ]:
# FEE PER DAY/in fixed USD value (total observation period)

#calculate USD value (define USD value, in this case locked to initial state observation time)
fee_t = p_df[['change_datetime']].dropna()
fee_t['DAI_fee_value'] = p_df.generated_fees_dai*1.0053414509361551 #define DAI price
fee_t['WETH_fee_value'] = p_df.generated_fees_weth*594.3526451552318 #define WETH price
daily = fee_t.groupby(pd.Grouper(key='change_datetime',freq='D')).sum() #sum per week

#plot value per day
fig = go.Figure(data=[go.Bar(name='DAI fees', x=daily.index, y=daily.DAI_fee_value, marker_color='#4675ed', offsetgroup=0), go.Bar(name='WETH fees', x=daily.index, y=daily.WETH_fee_value, marker_color='#4145ab', offsetgroup=1)], layout=go.Layout(title='<b>Fees per day</b> in USD', yaxis_title='USD value'))
fig.show()

### C1.4 Action Types

In [ ]:
# Action Types by proportion (total observation period)
actions = p_df.groupby('action_type').size().to_frame('count').reset_index()
fig = px.pie(actions, values='count', names='action_type', title='<b>Action Types</b>', hole=.3, color_discrete_sequence=px.colors.sequential.Turbo)
fig.show()
print(actions)

In [ ]:
#Action Type per timestep (total observation period)
fig = px.scatter(p_df, x='timestep', y='invariant', color='action_type', color_discrete_sequence=px.colors.sequential.Turbo, category_orders={'action_type': ['swap', 'external_price_update', 'join_swap', 'join', 'exit_swap', 'exit', 'pool_creation']})
fig.update_layout(height=400, width=1000, title_text="<b>Action Type / Timestep</b>")
fig.update_xaxes(rangeslider_visible=True)
fig.update_yaxes(range=[183000, 189000])
fig.show()